In [25]:
import json
import time
import pandas as pd
from collections import Counter
import torch

import pandas as pd
import numpy as np
import nltk
import string
import re
import emoji
import torch
import json
import time

from sklearn.utils import shuffle
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [26]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

[]


In [27]:
# Basic data files and paths
TOP_100_PATH = '../data/popular_100_memes.csv'
TOP_100 = pd.read_csv(TOP_100_PATH, encoding = "ISO-8859-1")
DATA_PATH = '../data/memes/'

STATS_PATH = '../data/statistics.json'
with open(STATS_PATH, 'r') as f:
    STATS = json.load(f)
print(TOP_100.head())

          ID                   Name  \
0  112126428   Distracted Boyfriend   
1     438680  Batman Slapping Robin   
2   87743020            Two Buttons   
3  181913649    Drake Hotline Bling   
4      61579    One Does Not Simply   

                                     Alternate Names  
0  distracted bf, guy checking out another girl, ...  
1                                                NaN  
2  2 red buttons, choice button, which button, da...  
3  drakeposting, drakepost, drake hotline approve...  
4  one does not simply walk into morder, lord of ...  


In [28]:
memes_count = Counter(STATS['memes'])
print(len(STATS['memes']))
for f, c in memes_count.most_common():
    print(f, c)

99
Bad-Luck-Brian.json 32141
One-Does-Not-Simply.json 26186
Philosoraptor.json 19971
Boardroom-Meeting-Suggestion.json 18164
Futurama-Fry.json 17325
First-World-Problems.json 17272
The-Most-Interesting-Man-In-The-World.json 15850
Batman-Slapping-Robin.json 14802
Bad-Pun-Dog.json 13194
X-X-Everywhere.json 12985
Expanding-Brain.json 12826
Two-Buttons.json 12232
But-Thats-None-Of-My-Business.json 11097
Ancient-Aliens.json 10555
Creepy-Condescending-Wonka.json 10374
Grumpy-Cat.json 9569
Y-U-No.json 9370
The-Rock-Driving.json 9286
That-Would-Be-Great.json 8600
Waiting-Skeleton.json 8592
Change-My-Mind.json 8563
Leonardo-Dicaprio-Cheers.json 8377
10-Guy.json 8258
Captain-Picard-Facepalm.json 8105
Matrix-Morpheus.json 7833
Third-World-Skeptical-Kid.json 7782
Confession-Bear.json 7778
Am-I-The-Only-One-Around-Here.json 7291
Success-Kid.json 7074
Hide-the-Pain-Harold.json 6973
Roll-Safe-Think-About-It.json 6948
Evil-Toddler.json 6106
Awkward-Moment-Sealion.json 6101
Face-You-Make-Robert-Downey-

In [29]:
selected_memes = [
    'Woman-Yelling-At-Cat.json',
    'Left-Exit-12-Off-Ramp.json',
    'Surprised-Pikachu.json',
    'Is-This-A-Pigeon.json',
    'Drake-Hotline-Bling.json'
]

def open_data(path):
    with open(path, 'r') as f:
        data = json.load(f)
    return data

memes_data = [open_data(DATA_PATH + meme) for meme in selected_memes]

In [30]:
# Preprocessing data for modelling
from nltk import sent_tokenize, word_tokenize
from gensim.models import Word2Vec

In [31]:
def parse_text(text):
    text = text.lower().strip()
    return word_tokenize(text)

def parse_blocks(blocks):
    result = []
    for i,b in enumerate(blocks):
        result.append(parse_text(b))
#         if i < len(blocks) - 1:
#             result.extend('|')
    return result

ex = parse_blocks(memes_data[0][0]['boxes'])
print(ex)
model = Word2Vec(ex, min_count=1)
print(model)

[['me', ':', 'i', 'do', "n't", 'know', 'what', 'to', 'do', 'with', 'my', 'life'], ['me', 'to', 'me', ':', 'and', 'i', 'do', "n't", 'want', 'to', 'see', 'it']]
Word2Vec(vocab=15, size=100, alpha=0.025)


In [32]:
# making X and y for RNN
word = []
y = []
for i in range(len(memes_data)):
    #print("This is i: " + str(i))
    for ele in memes_data[i]:
        y.append(i) 

        box = ele['boxes']
        parsedtext = parse_blocks(box)
        
        #appen all words in a box as to array2
        array2 = []
        for box_element in parsedtext:
            for box_word in box_element:
                array2.append(box_word)
        print(array2)        
        
        #each element in wordcontains all words of a box    
        word.append(array2)

X_words = word
print(word)
print(y) 

['me', ':', 'i', 'do', "n't", 'know', 'what', 'to', 'do', 'with', 'my', 'life', 'me', 'to', 'me', ':', 'and', 'i', 'do', "n't", 'want', 'to', 'see', 'it']
['you', 'said', 'you', 'were', 'a', 'black', 'person', 'i', 'lied']
['you', 'when', 'your', 'mom', 'yells', 'at', 'you', 'for', 'buying', 'robux', 'with', 'her', 'credit', 'card']
['study', 'for', 'test', 'study', 'for', 'everything']
['people', 'yelling', 'at', 'china', 'for', 'spreading', 'corona', 'china', 'testing', 'new', 'virus']
['me', 'thinking', 'a', 'demon', "'s", 'in', 'my', 'room', '.', 'pile', 'of', 'clothes', 'on', 'my', 'dresser', '.']
['the', 'earth', 'is', 'flat', 'shut', 'up', 'you', 'uncivilised', 'being']
['my', 'friend', 'shipping', 'me', 'and', 'some', 'other', 'dude', 'me', 'wondering', 'why', 'she', "'s", 'yelling']
['how', 'dare', 'you', 'cheat', 'on', 'me', '!', '!', 'dude', 'im', 'your', 'cat']
['we', 'are', 'not', 'naming', 'her', 'ripley', 'xenomorph', 'ripley']
['you', 'had', 'no', 'glove', 'on', 'your',

['video', 'games', 'cause', 'violence', 'goat', 'simulator']
['reddit', 'is', 'just', 'a', 'more', 'complicated', 'version', 'of', 'amino', 'amino', 'is', 'just', 'a', 'more', 'annoying', 'version', 'of', 'reddit']
['you', 'did', "n't", 'deserve', 'to', 'win', 'the', 'acatemy', 'award', '!', 'but', 'i', 'act', 'more', 'professionally', 'than', 'you', '.']
['you', 'said', 'you', 'hated', 'guns', 'and', 'they', 'stink', 'no', ',', 'i', 'said', 'guns', 'are', 'great', 'and', 'they', 'go', 'pew', 'pew', '.']
['dallas', 'cowboys', 'rule', 'they', 'suck', '!']
['fake', 'news', '!', 'real', 'as', 'your', 'american', 'titties']
['when', 'you', 'get', 'drunk', 'at', 'dinner', 'and', 'u', 'start', 'seeing', 'things', '.']
['your', 'mom', 'an', 'innocent', 'doughnut']
['stop', 'watching', 'youtube', 'tell', 'me', 'that', 'after', 'you', 'get', 'off', 'facebook']
['my', 'teacher', 'me', 'putting', 'memes', 'on', 'the', 'positivity', 'wall']
['i', "'m", 'going', 'on', 'a', 'carnival', 'cruise', '!'

['i', 'am', 'not', 'a', 'snowflake', '.', 'are', 'those', 'tears', 'or', 'are', 'you', 'melting', '?']
['``', 'toleit', 'paper', 'will', 'stop', 'covid-19', "''", 'im', 'just', 'going', 'to', 'get', 'food', ',', 'wather', ',', 'first', 'aid', 'stuff', 'then', 'tell', 'me', 'how', 'your', 'doing', '6', 'months', 'later', ',', 'ok']
['is', 'that', 'a', 'cat', '?', '!', 'no', ',', 'karen', ',', 'i', "'m", 'a', 'tiger']
['everyone', '2020']
['the', 'government', 'trying', 'steal', 'my', 'toilet', 'paper', 'me', ':']
['when', 'your', 'friend', 'looks', 'batter', 'than', 'you', 'sorry', 'not', 'sorry']
['the', 'downvotes', 'show', 'how', 'horrible', 'your', 'memes', 'are', 'the', 'downvotes', 'are', 'from', 'australians', 'who', 'tried', 'to', 'upvote']
['you', 'said', 'everything', 'would', 'be', 'free', '!', '!', '...', '..and', 'you', 'were', 'dumb', 'enough', 'to', 'believe', 'it', '!', '!', '!']
['the', 'weird', 'teacher', 'the', 'one', 'kid', 'with', 'a', 'hoodie']
['my', 'mom', 'at', 

['rollin', 'down', 'the', 'street', ',', 'smokin', 'indo', ',', 'sippin', 'on', 'gin', 'and', 'juice', '...', 'laid', 'back', '[', 'with', 'my', 'mind', 'on', 'my', 'money', 'and', 'my', 'money', 'on', 'my', 'mind', ']', 'where', 'were', 'you', '?', '~', '!']
['popeye', "'s", 'spicy', 'chicken', 'sandwich', 'chick', 'fil', 'a']
['america', 'in', '1846', ':', 'i', "'m", 'manifesting', 'my', 'destiny', 'mexico', 'just', 'trying', 'to', 'eat', 'dinner', ':', 'bruh', 'wut']
['anti-vaxxx', 'moms', 'saying', 'vaccines', 'cause', 'autisum', 'me', ',', 'an', 'intellectual', ':']
['people', 'saying', 'fnaf', 'is', 'demonic', '.', 'me', ',', 'a', 'fnaf', 'fan']
['wednesday', '!', 'hump', 'day', '.']
['you', 'are', 'not', 'sick', '!', 'i', 'have', 'a', 'case', 'of', 'corona', '.']
['i', "'m", 'not', 'obama', '!', 'nor', 'nancy', 'pelosi', '!', '!', 'your', 'impeached', '!', '!']
['i', 'said', 'turn', 'it', 'up', 'i', '’', 'll', 'turn', 'it', 'up', 'when', 'ronnie', 'van', 'zant', 'tells', 'me', '

['johnny', 'no', 'i', 'am', 'tommy']
['you', 'millenials', 'dont', 'understand', 'ok', 'boomer']
['mom', 'i', 'do', "n't", 'want', 'to', 'go', 'back', 'to', 'school', 'yes', 'you', 'do', '!']
['your', 'sugar', 'levels', 'are', 'too', 'high', '!', '!', '!', 'no', 'worries', '.', 'i', "'ll", 'put', 'the', 'tin', 'on', 'the', 'floor']
['you', 'said', 'you', 'love', 'me+me', 'only', 'yeah', 'meme']
['lieberry', 'it', "'s", 'called', 'a', 'library']
['my', 'mom', 'yelling', 'at', 'the', 'school', 'system', 'because', 'it', 'doesn', '’', 't', 'make', 'any', 'sense', 'me', 'trying', 'to', 'listen', 'to', 'a', 'youtube', 'video']
['my', 'wife', 'yelling', 'at', 'me', 'me', 'wondering', 'why', 'the', 'dishwasher', 'is', 'talking']
['paradise', 'city', '``', 'take', 'me', 'down', 'to', 'paradise', 'city', "''", '``', 'take', 'me', 'down', 'to', 'prairie-dog', 'city', "''"]
['happy', 'thanksgiving', '!', 'leave', 'me', 'out', 'of', 'the', 'group', 'text', '.']
['the', 'song', 'is', '“', 'let', 'i

...

['anti-vaxxers', 'sound', 'logic', 'is', 'this', 'autism', '?']
['when', 'i', 'study', 'my', 'memory', 'after', '3', 'seconds']
['essential', 'oils', 'anti', 'vaxxers', 'is', 'this', 'a', 'cure', '?']
['i', 'take', 'lax', 'toilets', 'clogged', 'i', 'poop', 'on', 'the', 'floor']
['me', 'money', 'is', 'it', 'tuesday', 'yet', '?']
['parents', 'anything', 'they', 'do', "n't", 'know', 'is', 'this', 'pg', '?']
['my', 'future', 'this', 'brick', 'is', 'in', 'my', 'path']
['izuku', 'todoroki', 'is', 'it', 'a', 'ship']
['im', 'murder', 'in', 'that', 'case', 'i', 'will', 'run', 'r.i.p']
['any', 'skeleton', 'is', 'this', 'an', 'undertale', '?', 'this', 'is', 'why', 'we', 'play', 'undertale', 'chummy', 'chum', 'chum']
['me', 'me', 'is', 'this', 'depression', '?']
['weebs', 'waifu', 'pillows', 'gim', 'me']
['memes', 'and', 'imgflip', 'raydog', 'is', 'this', 'what', 'everyone', 'calles', 'a', 'life', '?']
['is', 'this', 'a', 'pigeon', '?']
['me', 'my', 'meme', 'is', 'this', 'a', 'good', 'meme', '?']


['is', 'this', 'where', 'i', 'say', 'yikes', '?', 'me', 'a', 'bad', 'situation', 'i', 'created']
['me', 'cancer', 'is', 'this', 'a', 'virus']
['is', 'this', 'a', 'pigeon', '?', 'no', 'it', "'s", 'a', 'self', 'aware', 'meme']
['anata', 'wa', 'sudeni', 'shinde', 'iru', 'nani', '!', '?', '!', '?', '!', '?', '!', '?', '!', 'anime', ',', 'am', 'i', 'right', '?']
['hey', 'bae', 'do', "n't", 'tell', 'my', 'gf', 'that', 'we', "'re", 'low', 'key', 'dating', 'she', 'paid', 'me', '$', '180', 'to', 'say', 'the', 'truth', 'i', "'m", 'sorry', 'bae', 'its', 'not', 'a', 'big', 'deal', 'i', 'swear', 'them', 'hoe', "'s", 'ai', "n't", 'loyal']
['logan', 'paul', 'tasing', 'a', 'dead', 'rat', 'is', 'this', 'content']
['is', 'this', 'a', 'waste', 'of', 'my', 'time', 'detective', 'pikachu', 'this', 'movie', 'makes', 'no', 'sense']
['moving', 'butt', 'white', 'person', 'is', 'this', 'a', 'twerk']
['12', 'year', 'olds', 'free', 'vbuck', 'scams', 'is', 'this', 'a', 'vbuck', 'generator', '?']
['this', 'is', 'a',

['using', 'drake', 'meme', 'format', 'using', 'smm2', 'meme', 'format', 'to', 'get', 'upvotes', 'on', 'reddit']
['coronavirus', 'boomer', 'remover']
['teacher', 'when', 'the', 'boys', 'at', 'the', 'back', 'are', 'talking', 'when', 'i', 'ask', 'for', 'a', 'pencil']
['calling', 'in', 'sick', 'calling', 'in', 'coronavirus']
['learning', 'math', 'when', 'your', 'teacher', "'s", 'in', 'a', 'bad', 'mood', '.', 'learning', 'math', 'with', 'an', 'idiot', 'sub', '.']
['getting', 'my', 'toes', 'licked', 'getting', 'my', 'booty', 'hole', 'licked']
['i', 'have', 'to', 'go', 'to', 'the', 'bathroom', 'i', 'have', 'to', 'crap']
['other', 'robotics', 'institute', 'robo', 'inventors']
['when', 'she', 'get', 'the', 'corronaviris', 'when', 'i', 'get', 'the', 'coronavirus']
['valentine', '’', 's', 'day', 'day', 'to', 'make', 'people', 'feel', 'bad', 'about', 'how', 'lonely', 'they', 'are']
['8', 'hours', 'of', 'sleep', '3', 'hours', 'of', 'sleep']
['sharing', 'facebook', 'memes', 'making', 'your', 'own', 

['256', 'block', 'height', 'limit', '4', 'stacks', 'of', 'blocks', 'height', 'limit']
['digital', 'piracy', 'getting', 'permission', 'from', 'copywright', 'holders', 'to', 'duplicate', 'and', 'share', 'another', 'person', "'s", 'work']
['turkey', 'bacon', 'regular', 'bacon']
['robux', 'generators', 'claimrbx']
['overly', 'happy', 'and', 'excited', 'high', 'on', 'crack']
['when', 'you', 'have', '200', 'upvotes', 'when', 'you', 'have', '500', 'upvotes']
['guacamole', 'avacado', 'sauce']
['5,000,000', 'views', ',', '1,000,000', 'likes', ',', 'and', '1,000,000', 'comments', '1,000,001']
['boy', 'boi']
['actually', 'raiding', 'area', '51', 'camping', 'in', 'front', 'of', 'area', '51']
['having', 'a', 'girlfriend', 'playing', 'video', 'games']
['ultamite', 'dis', 'track', 'darkest', 'hour']
['doing', 'long', 'term', 'assignments', 'over', 'two', 'weeks', 'doing', 'long', 'term', 'assignments', 'in', 'one', 'night']
['invasion', 'of', 'haiti', 'temporary', 'military', 'occupation', 'to', 'pre

['infantos', 'iaceo', 'infantos', 'iacio']
['wearing', 'socks', 'with', 'sandals', ':', '(', 'wearing', 'sandals', 'with', 'socks', ':', ')']
['using', 'other', 'formats', 'using', 'drake', 'format', 'because', 'the', 'other', 'formats', 'of', 'this', 'meme', 'made', 'the', 'original', 'drake', 'format', 'die']
['ww3', 'w-2']
['due', 'tomorrow', 'do', 'tomorrow']
['while', 'reading', 'ready', 'player', 'one', 'misspelling', 'art3mis', "'", 'name', 'as', 'art-three-miss', 'until', 'she', 'corrects', 'you', 'continuing', 'to', 'say', 'her', 'name', 'art-three-miss', 'even', 'after', 'her', 'correction']
['android', 'ios']
['apple', 'doctor', 'repellent']
['yeet', '!', '!', '!', '!', 'get', 'nae', 'naed']
['hack', 'minecraft', 'open', 'create', 'mode']
['walmart', 'order', 'online']
['not', 'knowing', 'what', 'to', 'do', 'for', 'a', 'meme', 'doing', 'what', 'everyone', 'else', 'is', 'doing']
['ok', 'boomer', 'boomer', 'ok']
['santa', 'claus', 'baby', 'yoda', 'as', 'santa']
['$', '10', '$'

In [33]:
X_words[0]

['me',
 ':',
 'i',
 'do',
 "n't",
 'know',
 'what',
 'to',
 'do',
 'with',
 'my',
 'life',
 'me',
 'to',
 'me',
 ':',
 'and',
 'i',
 'do',
 "n't",
 'want',
 'to',
 'see',
 'it']

In [14]:
len(y)

18367

In [16]:
len(X_words)

18367

In [15]:
yset = set(y)
yset

{0, 1, 2, 3, 4}

In [34]:
# making word embeddings for RNN
UNK = "<unk>"
WORDEMBSIZE = 64
W2V_WINDOW = 7
W2V_COUNT = 1
W2V_EPOCH=100

def makeVocab(text):
    vocab = set()
    for sentences in text:
        for word in sentences:
            vocab.add(word)
    return vocab

def makeEmbModel(data):
    docs = [[UNK]]
    docs.extend(data)
    model = Word2Vec(window=W2V_WINDOW, min_count=W2V_COUNT, size=WORDEMBSIZE)
    model.build_vocab(docs)
    model.train(docs, total_examples=len(docs), epochs=W2V_EPOCH)
    print(model)
    return model

def makeEmbeddings(data, model, vocab):
    vecData = []
    for sentence in data:
        wordEmbs = []
        for word in sentence:
            if word in vocab:
#                 print(type(model[word]))
#                 print(model[word])
                wordEmbs.append(model[word])
            else:
                wordEmbs.append(np.zeros(WORDEMBSIZE))
        wordEmbs = torch.FloatTensor(wordEmbs)
        vecData.append(wordEmbs)
    return vecData

In [35]:
# continue making word embeddings
X_words_train, X_words_test, y_train, y_test = train_test_split(X_words, y, test_size=0.2, random_state=42)
vocab = makeVocab(X_words_train)
emb_model = makeEmbModel(X_words_train)
vocab = set(list(emb_model.wv.vocab.keys()))
X_train = makeEmbeddings(X_words_train, emb_model, vocab)
X_test = makeEmbeddings(X_words_test, emb_model, vocab)

print(X_words_train[0])
print(X_train[0])
print(y_train[0])

Word2Vec(vocab=17434, size=64, alpha=0.025)


/Users/kathy908000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


['belt', 'ww3', 'me']
tensor([[ 0.6852,  0.6181, -0.2568,  0.3250, -1.3172,  0.2496,  1.3115, -0.6972,
         -0.4141,  0.5243, -1.7616,  0.2553, -0.6701,  1.6186,  0.3276,  0.8346,
         -0.9366, -1.9377,  2.3634, -0.3053, -0.5340,  0.5596, -0.2272, -2.2908,
         -0.4170, -2.3659,  1.3769,  1.5357,  0.0102,  0.9227, -0.6851,  0.5771,
          2.0212,  0.8715,  0.4708, -0.4264,  3.3132,  0.4821, -0.6754, -1.1405,
          0.4538,  0.7242, -0.1118, -1.6256, -0.1296, -0.7650, -0.0934, -1.6463,
         -1.5256, -0.5843,  0.8034, -0.2167,  0.0690,  0.4020,  0.6104, -0.5651,
          0.3560, -0.5685,  0.4722, -1.9669,  1.1146, -1.0029, -1.3665,  1.2768],
        [-0.4765,  0.5289,  1.1611, -0.8996,  1.0058,  0.0714,  0.8915,  0.4927,
          0.8312,  0.5370, -0.6288,  0.9239, -0.9126,  0.2235, -0.4757,  0.7656,
          0.2865, -1.8399,  1.2815,  0.3319, -0.1546, -1.0035,  0.0527,  0.0968,
         -0.2781, -0.6563, -0.1969,  0.5809, -0.7759,  1.6513, -1.8309,  0.1211,
     

In [36]:
# RNN building to predict result
import torch
import torch.nn as nn
import torch.optim as optim
import random
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [37]:
# RNN model
class RNN(nn.Module):
    def __init__(self, input_dim, h, output_dim = 5):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_dim, h, dropout=0.2)
        self.finalLayer = nn.Linear(h, output_dim)
        self.input_dim = input_dim
        self.hidden_dim = h
        self.output_dim =  output_dim
#         self.word_embeddings = nn.Embedding(vocab_size, input_dim)
        self.softmax = nn.LogSoftmax()
        self.loss = nn.NLLLoss()
    
    def compute_loss(self, predicted_vector, gold_label):
        return self.loss(predicted_vector, gold_label)
    
    def forward(self, inputs):
        out, hidden = self.rnn(inputs)
        hidden = hidden.contiguous().view(-1,self.hidden_dim)
        predicted_vector = self.softmax(self.finalLayer(hidden))
        return predicted_vector

In [38]:
# running epochs for training and validation
HIDDEN_DIM = 128
EPOCHS = 30

model = RNN(WORDEMBSIZE, HIDDEN_DIM, 5)
optimizer = optim.SGD(model.parameters(),lr=0.0005, momentum=0.9)
train_loss = []
test_loss = []
for epoch in range(EPOCHS):
    print("\n\n-------------")
    print("EPOCH: {}".format(epoch + 1))
    running_loss = 0.0
    model.train()
    optimizer.zero_grad()
    print("Training started for epoch:{}".format(epoch + 1))
    X_train, y_train = shuffle(X_train, y_train)
    start_time = time.time()
    correct = total = 0
    minibatch_size = 16
    N = len(y_train)
    for minibatch_idx in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        loss = None
        for idx in range(minibatch_size):
            text = X_train[minibatch_idx * minibatch_size + idx]
            text = torch.unsqueeze(text, 1)
            labelIdx = y_train[minibatch_idx * minibatch_size + idx]
            log_probs = model(text)
            text_loss = model.compute_loss(log_probs.view(1,-1), torch.tensor([labelIdx]))
            running_loss += text_loss
            if loss is None:
                loss = text_loss
            else:
                loss += text_loss
            pred_label = torch.argmax(log_probs)
            correct += int(pred_label == labelIdx)
            total += 1
        loss = loss / minibatch_size
        loss.backward()
        optimizer.step()
    train_loss.append(running_loss / N)
    print("Training completed for epoch: {}".format(epoch + 1))
    print("Time for train: {}".format(time.time() - start_time))
    print("Accuracy: {}".format(correct / total))
    
    #validation
    running_loss = 0.0
    model.eval()
    optimizer.zero_grad()
    print("Validation started for epoch:{}".format(epoch + 1))
    X_test, y_test = shuffle(X_test, y_test)
    start_time = time.time()
    correct = total = 0
    minibatch_size = 16
    N = len(y_test)
    for minibatch_idx in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        for idx in range(minibatch_size):
            text = X_test[minibatch_idx * minibatch_size + idx]
            text = torch.unsqueeze(text, 1)
            labelIdx = y_test[minibatch_idx * minibatch_size + idx]
            log_probs = model(text)
            text_loss = model.compute_loss(log_probs.view(1,-1), torch.tensor([labelIdx]))
            running_loss += text_loss
            pred_label = torch.argmax(log_probs)
            correct += int(pred_label == labelIdx)
            total += 1
    test_loss.append(running_loss / N)
    print("Validation completed for epoch: {}".format(epoch + 1))
    print("Time for validation: {}".format(time.time() - start_time))
    print("Accuracy: {}".format(correct / total))

plt.plot(train_loss, label='train loss')
plt.plot(test_loss, label='test loss')
plt.legend()
plt.show()



-------------
EPOCH: 1


/Users/kathy908000/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Training started for epoch:1


/Users/kathy908000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: input must have 3 dimensions, got 2

In [ ]:
selected_memes = [
    'Woman-Yelling-At-Cat.json',  #4604
    'Left-Exit-12-Off-Ramp.json', #4319
    'Surprised-Pikachu.json',    #3507
    'Is-This-A-Pigeon.json',   #3507
    'Drake-Hotline-Bling.json',  #2690
    'Blank-Nut-Button.json', #5714
    'One-Does-Not-Simply.json',# 26186 #only use 5000 to match button meme number
    'Change-My-Mind.json', #8563
    'Roll-Safe-Think-About-It.json', #6948
    'Leonardo-Dicaprio-Cheers.json' #8377
]